In [5]:
import json
import pandas as pd

with open("./data/users.json", "r") as f:
    users = json.load(f)
    
with open("./data/debates.json", "r") as f:
    all_debates = json.load(f)
    
for key, value in users.items():
    value["name"] = key

In [6]:
number_of_users=len(users)
# number_of_users
print(users["ahuggies30"].keys())
# print(len(users["ahuggies30"].keys()))

# categories = ["birthday","education","ethnicity","gender","looking","number_of_friends","party","political_ideology","president","relationship","religious_ideology"]
# print(len(categories))

dict_keys(['all_debates', 'big_issues_dict', 'birthday', 'description', 'education', 'elo_ranking', 'email', 'ethnicity', 'gender', 'friends', 'income', 'interested', 'joined', 'last_online', 'last_updated', 'looking', 'lost_debates', 'number_of_all_debates', 'number_of_lost_debates', 'number_of_tied_debates', 'number_of_won_debates', 'number_of_friends', 'number_of_opinion_arguments', 'number_of_opinion_questions', 'number_of_poll_topics', 'number_of_poll_votes', 'number_of_voted_debates', 'opinion_arguments', 'opinion_questions', 'party', 'percentile', 'political_ideology', 'poll_topics', 'poll_votes', 'president', 'relationship', 'religious_ideology', 'url', 'voted_debates', 'win_ratio', 'won_debates', 'tied_debates', 'name'])


In [48]:
# list_users = list(users.values())
print(list_users[2]["big_issues_dict"])

{'Abortion': 'Pro', 'Affirmative Action': 'N/O', 'Animal Rights': 'Pro', 'Barack Obama': 'Pro', 'Border Fence': 'N/O', 'Capitalism': 'Pro', 'Civil Unions': 'N/O', 'Death Penalty': 'Und', 'Drug Legalization': 'Und', 'Electoral College': 'Und', 'Environmental Protection': 'Pro', 'Estate Tax': 'N/O', 'European Union': 'Pro', 'Euthanasia': 'Pro', 'Federal Reserve': 'Pro', 'Flat Tax': 'Con', 'Free Trade': 'Pro', 'Gay Marriage': 'Pro', 'Global Warming Exists': 'Pro', 'Globalization': 'Pro', 'Gold Standard': 'Con', 'Gun Rights': 'Con', 'Homeschooling': 'N/O', 'Internet Censorship': 'Con', 'Iran-Iraq War': 'Con', 'Labor Union': 'Pro', 'Legalized Prostitution': 'Pro', 'Medicaid & Medicare': 'N/O', 'Medical Marijuana': 'Pro', 'Military Intervention': 'Pro', 'Minimum Wage': 'Pro', 'National Health Care': 'Pro', 'National Retail Sales Tax': 'N/O', 'Occupy Movement': 'Con', 'Progressive Tax': 'Pro', 'Racial Profiling': 'Und', 'Redistribution': 'Pro', 'Smoking Ban': 'Con', 'Social Programs': 'Pro', 

In [8]:
# print(list(users["ahuggies30"]["big_issues_dict"].values()))
# print(len(users["ahuggies30"]["big_issues_dict"].keys()))

In [9]:
print(users["ahuggies30"]['all_debates'])

['Dogs-are-better-than-cats/28/', 'rainbows-dnt-exist-if-you-are-blind/1/', 'EA-worst-gaming-company/1/', 'Is-soccer-a-sport/1/']


In [38]:
def get_debate_text(user, all_debates):
    name = user["name"]
    debates = []

    debates_names = user["all_debates"]
    for debate_name in debates_names:
        if all_debates.get(debate_name):
            this_debate = all_debates[debate_name]
            position = ""
            if this_debate["participant_1_name"] == name:
                position = this_debate["participant_1_position"]
            else:
                position = this_debate["participant_2_position"]

            for round in this_debate["rounds"]:
                for argument in round:
                    if argument["side"] == position:
                        text = argument["text"].replace("\n","").replace("\r","").replace("\t","").lstrip().rstrip()
                        text += " "
                        debates.append(text)

    return debates

def get_opinion_text(user):
    opinions = []
    arguments = user["opinion_arguments"]
    
    for argument in arguments:
        text = argument['opinion text'].replace("\n","").replace("\r","").replace("\t","").lstrip().rstrip()
        text += " "
        opinions.append(text)
        
    return opinions

In [49]:
# Here we convert the debate dataset into a desired form for the model
user_collection = []
cats = list(users["ahuggies30"].keys())
useful_cats= ['political_ideology', 'education', 'ethnicity', 'interested', 'gender' , 'religious_ideology']

per_cat_choices = dict()
for cat in useful_cats:
    per_cat_choices[cat] = []

for name,user in users.items():
    user_data = dict()
    user_data["name"] = (name)
    
    # categorical data
    for cat in useful_cats:
        user_data[cat] = user[cat]

        if user[cat] not in per_cat_choices[cat]:
            per_cat_choices[cat].append(user[cat])
            
    # text 
    debates = get_debate_text(user, all_debates)
    opinions = get_opinion_text(user)
    
    user_data["debates"] = debates
    user_data["opinions"] = opinions
    
    user_data["big_issues_dict"] = user["big_issues_dict"]
    
    if not (len(debates) == 0 and len(opinions) == 0):
        user_collection.append(user_data)
    
# prepare one-hot
for user_data in user_collection:
    for cat in useful_cats:
        user_data[cat + "_id"] = per_cat_choices[cat].index(user_data[cat])
        user_data[cat + "_len"] = len(per_cat_choices[cat])

In [50]:
print(len(user_collection))

37075


In [51]:
print(user_collection[:10])

[{'name': '00003', 'political_ideology': 'Not Saying', 'education': 'Not Saying', 'ethnicity': 'Not Saying', 'interested': 'No Answer', 'gender': 'Prefer not to say', 'religious_ideology': 'Not Saying', 'debates': [], 'opinions': ["Only conservatives would support him.  I don't consider myself a liberal/leftist/democrat/far-left person when it comes to politics but Ben Shapiro wouldn't be a very good president. He would make a terrible foreign policy and wouldn't be good for the transgender community. He seems like he's probably kind as a person, but i don't agree with all of his politics. ", "It's a terrible idea.  What if the teacher is a pacifist or is afraid of using guns or doesn't know how to use guns or whatever? There's a teacher who shot a gun in a school recently and nobody else was shooting. This shows that some teachers could become school shooters themselves. Their guns could make students feel threatened. Besides, a student could suddenly steal the gun from the teacher an

In [47]:
# saving
with open("text_debate_users.json","w",encoding="UTF-8") as f:
    json.dump(user_collection,f)

In [58]:
# # Preparing TSV
# data = []
# headers = ["text"]
# headers.extend(list(all_users["ahuggies30"]["big_issues_dict"].keys()))


# for user in users.values():
#     temp = []
#     text = ""
    
#     debates = get_debate_text(user, all_debates)
    
#     for debate in debates:
#         text += debate
#         text += " "
    
#     text = text.replace("\n"," ")
#     text = text.replace("\t"," ")
#     text = text.replace("\r","")
#     text = text.replace("  ","")
    
#     if len(text)>0:
#         temp.append(text)
#     else:
#         continue
    
#     labels = []
#     attitudes = list(user["big_issues_dict"].values())
#     mask = ["Pro","Con","Und","N/O","N/S"]
    
#     for attitude in attitudes:
#         labels.append(mask.index(attitude))
    
#     temp.extend(labels)
#     data.append(temp)

# df = pd.DataFrame(data, columns=headers)   

In [ ]:
cat_vec, ling_vec, tokenized_sents, task_embs